### VOC格式
- .xml
- 原始坐标
- 左上角 + 右下角

In [1]:
from xml.etree import ElementTree

In [3]:
tree = ElementTree.parse(source="0000001.xml")

In [4]:
root = tree.getroot()

In [8]:
img_path = root.find(path="path").text

In [9]:
img_path

'C:\\Users\\pkulx\\Desktop\\day14\\label_demo\\images\\0000001.jpg'

In [17]:
img_width = int(root.find(path="size").find(path="width").text)
img_height = int(root.find(path="size").find(path="height").text)

In [18]:
img_height, img_width

(1500, 1500)

In [27]:
for obj in root.findall(path="object"):
    print(obj.find("name").text)
    xmin = int(obj.find("bndbox").find("xmin").text)
    ymin = int(obj.find("bndbox").find("ymin").text)
    xmax = int(obj.find("bndbox").find("xmax").text)
    ymax = int(obj.find("bndbox").find("ymax").text)
    print(xmin, ymin, xmax, ymax)
    print("-" * 80)

A330
639 112 825 304
--------------------------------------------------------------------------------
ARJ21
241 786 422 967
--------------------------------------------------------------------------------
Boeing787
933 750 1122 973
--------------------------------------------------------------------------------
other
158 90 329 275
--------------------------------------------------------------------------------


In [30]:
label2idx = {"A330": 0, 
             "A320/321": 1, 
             "A220": 2, 
             "ARJ21":3, 
             "Boeing737": 4, 
             "Boeing787":5, 
             "other":6}

In [31]:
idx2label = {idx: label for label, idx in label2idx.items()}

In [32]:
idx2label

{0: 'A330',
 1: 'A320/321',
 2: 'A220',
 3: 'ARJ21',
 4: 'Boeing737',
 5: 'Boeing787',
 6: 'other'}

In [41]:
def transfer_yolo(file_name="./0000001.xml"):
    """
     把VOC格式转为yolo格式
    """
    tree = ElementTree.parse(source=file_name)
    root = tree.getroot()
    # 图像宽度和高度
    img_width = int(root.find(path="size").find(path="width").text)
    img_height = int(root.find(path="size").find(path="height").text)
    with open(file=file_name.replace(".xml", ".txt"), mode="w", encoding="utf8") as f:
        # 遍历每个目标
        for obj in root.findall(path="object"):
            name = obj.find("name").text
            cls_id = label2idx.get(name)
            xmin = int(obj.find("bndbox").find("xmin").text)
            ymin = int(obj.find("bndbox").find("ymin").text)
            xmax = int(obj.find("bndbox").find("xmax").text)
            ymax = int(obj.find("bndbox").find("ymax").text)
            # 中心点x
            x_center = round(number=(xmin + xmax) / 2 / img_width, ndigits=6)
            y_center = round(number=(ymin + ymax) / 2 / img_height, ndigits=6)
            box_width = round(number=(xmax - xmin) / img_width, ndigits=6)
            box_height = round(number=(ymax - ymin) / img_height, ndigits=6)
            print(cls_id, x_center, y_center, box_width, box_height, sep=" ", end="\n", file=f)

In [42]:
transfer_yolo()

### 2. COCO

In [43]:
import json

In [46]:
with open(file="./0000001.json", mode="r", encoding="utf8") as f:
    data = json.loads(s=f.read())

In [50]:
objs = data[0]

In [54]:
objs.keys()

dict_keys(['image', 'annotations'])

In [55]:
objs["image"]

'0000001.jpg'

In [60]:
for obj in objs["annotations"]:
    label = obj["label"]
    coordinates = obj["coordinates"]
    print(label)
    print(coordinates)

A330
{'x': 732.5, 'y': 208.0769230769231, 'width': 186.0, 'height': 192.0}
ARJ21
{'x': 332.0, 'y': 876.5769230769231, 'width': 181.0, 'height': 181.0}
Boeing787
{'x': 1028.0, 'y': 861.5769230769231, 'width': 189.0, 'height': 223.0}
other
{'x': 244.0, 'y': 182.5769230769231, 'width': 171.0, 'height': 185.0}
